# MS141 Lecture 14 

# Case study: Hyperbolic PDEs

## Read: Chapter 9 of Newman's book.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import sys

%matplotlib inline

# Set common figure parameters
newparams = {'figure.figsize': (16, 8), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 24}
plt.rcParams.update(newparams)

# Dictionary for line colors for different solvers
new_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']

color_dict = {}
color_dict['Exact']  = new_colors[1]
color_dict['FTCS']   = new_colors[0]
color_dict['Upwind']   = new_colors[2]
color_dict['PC-CS']  = 'blue'
color_dict['PC-FS']  = 'green'
color_dict['PC-BS']  = 'black'

## Advection equation

Let us consider the advection equation (of the hyperbolic family)

$$~~\frac{\partial u(x,t)}{\partial t} + a \frac{\partial u(x,t)}{\partial x} = 0.~~~~$$

For this equation, the exact analytical solution is known:

$$ u(x,t) = u(x-at,0). $$

This solution is obtained by rigidly shifting the function at $t=0$, $u(x,t=0)$. If $a>0$, the solution is moving along the $x$ axis from left to right. We will study numerical approaches to solve the advection equation and discuss their stability, using the known exact solution as a reference.<br> 

Note that even though the exact solution is known in principle, often advection is accompanied by collisions (e.g., in the Boltzmann equation), and thus it is only part of the problem. In addition, for given regular space and time grids, it is usually not possible to represent exactly $u(x-at,0)$, especially in more than one dimension. One would need at least an approach to reliably interpolate it on the chosen grid at each time step.

### Discretization
Similar to the previous lectures, we will solve the advection equation numerically using the finite difference method. We define the space and time grids:

$$\begin{align}
x_i&=x_{min}+i\Delta x \\
t_n&=n\Delta t,
\end{align}
$$

where $\Delta x$ and $\Delta t$ are the space and time steps, respectively. The discretization of the function $u(x,t)$ becomes:

$$ u(x,t) = u(x_i,t_n) = u^n_i, $$

where the subscript $i$ stands for the spatial coordinate and the superscript $n$ for time. We solve the equation on the spatial domain $x\in[0,1]$ ($x_{min}=0,x_{max}=1$), applying periodic boundary conditions:

$$u(0,t) = u(1,t).$$

### Forward-Time Centered-Space (FTCS) scheme 
Let us first try the FTCS scheme to discretize the advection equation. We use the explicit forward time derivative as in Euler's method:

$$ \frac{\partial u(x,t)}{\partial t} \approx \frac{u(x,t+\Delta t)-u(x,t)}{\Delta t} 
\implies \frac{\partial u(x,t)}{\partial t} \approx \frac{u_i^{n+1} - u_i^n}{\Delta t} + \mathcal{O}(\Delta t)$$

together with the centered spatial derivative: 

$$ \frac{\partial u(x,t)}{\partial x} \approx \frac{u(x+\Delta x,t)-u(x-\Delta x,t)}{2\Delta x} + \mathcal{O}(\Delta x^2) = \frac{u_{i+1}^{n}-u_{i-1}^n}{2\Delta x} + \mathcal{O}(\Delta x^2). $$

Therefore, the FTCS scheme is first-order in time and second-order in space.

We rewrite the advection equation in the FTCS discretized form

$$ \frac{u_i^{n+1}-u_i^n}{\Delta t} + a\frac{u_{i+1}^{n}-u_{i-1}^n}{2\Delta x} = 0, $$

and express the function at time $t_n+\Delta t$,  $u_i^{n+1}$, using the variables at time $t_n$:

$$ u_i^{n+1} = u_i^n - \frac{C}{2}(u^n_{i+1} - u^n_{i-1}),$$

where we defined the Courant number as $C \equiv a\Delta t/\Delta x$. Note that the definition of the Courant number for the advection equation is different from the heat equation discussed in the previous lecture. The advancing scheme defined by the last equation is *explicit* because one can directly find the values of the function at the next time step using only the function $u^n_i$ at the current time step. 

Let us now apply the FTCS approach with the initial condition for $u(x,t)$ equal to a Gaussian centered at $x=x_0$:

$$ u(x,t=0) = \exp{\{-\alpha (x-x_0)^2}\}. $$

We first define the necessary functions and space/time grids.

In [ ]:
# Gaussian initial condition
def initial_condition(x, alpha=50.0, x0=0.5):
    u = np.exp(-alpha * (x-x0)**2)
    return u

# Exact solution of the advection equation
def exact_solution(x, t, a):
    x_shift = np.remainder(x - a * t, xmax)
    return initial_condition(x_shift)

# FTCS solver
def ftcs(u, C):
    unew = np.zeros_like(u)
    # loop over space index i from 1 to Nx-1
    for i in range(1,u.shape[0]-1):
        unew[i] = u[i] - C / 2.0 * (u[i+1]-u[i-1])
                        
    # Periodic boundary conditions
    unew[0] = u[0] - C / 2.0 * (u[1] - u[-2])
    unew[-1] = unew[0]
    return unew

# Setting the 1D space grid
xmin = 0.0
xmax = 1.0
dx = 1e-2
Nx = int((xmax - xmin) / dx)
x = np.linspace(0, 1, Nx + 1)

# Setting the time grid
time_max = 10.0
dt = 1e-3
Nt = int(time_max/dt)   # 10000 steps                    
time_array = np.linspace(0,time_max,Nt+1)

# Courant number
a = 1.0
C = a*dt/dx # over time dt, wave advances by C spatial steps of size dx
print("Courant number C =", C)

Once the functions and grids are defined, we initialize the function $u(x,t)$, iterate over time, and plot the solution.

In [ ]:
# Array for the exact solution. 
u_exact = np.zeros([Nt+1, Nx+1])

# Initial condition for the exact solution
u_exact[0,:] = initial_condition(x)

# Initialization and initial condition for the FTCS solution
u_ftcs = np.zeros([Nt+1, Nx+1])
u_ftcs[0,:] = initial_condition(x)

# Iteration over time
for n in range(Nt):
    time = n * dt
    u_ftcs[n+1,:] = ftcs(u_ftcs[n,:], C)
    u_exact[n+1,:] = exact_solution(x, time, a)

    
    
# Function to make a plot
def plot_ax(ax, x, u, u_exact, label, title=None):
    ax.plot(x,u_exact,label='Exact', color=color_dict['Exact'])
    ax.plot(x,u,label=label, color = color_dict['FTCS'])
    ax.legend(loc='upper right')
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    ax.set_xlim([0, 1])
    if title:
        ax.set_title(title)

fig, axes = plt.subplots(ncols=2)

# Plot the solution for two times: t1 = 2.8 and t2 = 4.0

ntime1 = int(2.8 / dt)
title = 'Time = '+str(time_array[ntime1])+'; C = '+str(C)
plot_ax(axes[0], x, u_ftcs[ntime1,:], u_exact[ntime1,:], 'FTCS', title = title)

ntime2 = int(4.0 / dt)
title = 'Time = '+str(time_array[ntime2])+'; C = '+str(C)
plot_ax(axes[1], x, u_ftcs[ntime2,:], u_exact[ntime2,:], 'FTCS', title = title)

plt.tight_layout()
plt.show()

The FTCS is unstable and rapidly diverges. To characterize the deviation between the numerical solution $u(x,t)$ and the exact solution $u_{\rm ex}(x,t)$ as a function of time, we compute the root-mean-square error (RMSE):

$$ {\rm RMSE}(t)= \sqrt{ \frac{1}{N_x} \sum_{i=1}^{N_x} \big( u(x_i,t) - u_{\rm ex}(x_i,t) \big) }.$$

In [ ]:
def rmse(u,u_exact):
    rmse = np.zeros(u.shape[0],dtype=float)
    for i in range(u.shape[0]):
        rmse[i] = np.sqrt( ((u[i,:] - u_exact[i,:]) ** 2).mean() )
    return rmse

Now we can plot the deviation of the FTCS solution as a function of time

In [ ]:
rmse_ftcs = rmse(u_ftcs,u_exact)

plt.figure(figsize=(12,4))
plt.plot(time_array, rmse_ftcs, color = color_dict['FTCS'])
plt.xlabel('time')
plt.ylabel('RMSE')
plt.xlim([0,4])
plt.ylim([-1,30])
plt.show()

The FTCS solution begins to diverge at $t\approx3.0$. By decreasing the time step $dt$, one can change the time for the onset of the divergence, but the solution will still be unstable and will ultimately diverge.

### Upwind scheme

As the solution provided by the FTCS scheme is unstable, we try to change the spatial derivative while keeping the forward Euler time derivative to obtain a new explicit method. As we discussed above, if $a$ is positive, the exact solution is a rigid wave form moving from left to right (i.e., toward larger values of $x$). Therefore, when $a>0$, the left direction points "upstream" and is called the upwind direction. Biasing the derivative in the upwind direction, and thus taking a backward derivative, gives a stable method known as the [upwind scheme](https://en.wikipedia.org/wiki/Upwind_scheme) (alternatively, FTBS). 

The upwind (backward) spatial derivative is defined as:

$$ \frac{\partial u(x,t)}{\partial x} \approx \frac{u_{i}^{n}-u_{i-1}^n}{\Delta x}. $$

Therefore, the upwind solution scheme becomes:

$$ u_i^{n+1} = u_i^n - C(u^n_{i} - u^n_{i-1}).$$

While the upwind method is conditionally stable (see below), it is only first-order in both the space and time variables, and thus is not expected to be very accurate. Let us implement the upwind solver and verify its stability.

In [ ]:
# Upwind solver
def upwind(u, C):
    unew = np.zeros_like(u)
    # loop over space index from 1 to Nx-1
    for i in range(1,u.shape[0]-1):
        unew[i] = u[i] - C * (u[i]-u[i-1])
                        
    # Periodic boundary conditions
    unew[0] = u[0] - C * (u[0] - u[-2])
    unew[-1] = unew[0]
    return unew

# Initialization and initial condition 
u_up = np.zeros([Nt+1, Nx+1])
u_up[0,:] = initial_condition(x)

# Iteration over time
for n in range(Nt):
    time = n * dt
    u_up[n+1,:] = upwind(u_up[n,:], C)

In [ ]:
# plot the solution at two times
fig, axes = plt.subplots(ncols=2)

# Plot the solution for two times: t1 = 2.8 and t2 = 4.0

ntime1 = int(0.2 / dt)
title = 'Time = '+str(time_array[ntime1])+'; C = '+str(C)
plot_ax(axes[0], x, u_up[ntime1,:], u_exact[ntime1,:], 'Upwind', title = title)
axes[0].legend(loc='upper left')

ntime2 = int(1.8 / dt)
title = 'Time = '+str(time_array[ntime2])+'; C = '+str(C)
plot_ax(axes[1], x, u_up[ntime2,:], u_exact[ntime2,:], 'Upwind', title = title)

plt.tight_layout()
plt.show()

The upwind solution is stable but over time the shape of $u(x,t=0)$ changes due to the truncation error. It can be shown that the error introduces an artificial diffusion term proportional to $\Delta t^2$, which can be kept under control by choosing a small enough time step but cannot be fully removed.  

## Von Neumann stability analysis

We show a more formal stability analysis, first introduced by Von Neumann at Los Alamos during WWII. To analyze the stability of the advection equation solutions, we express $u(x,t)$ as the Fourier transform over the spatial variable at each time:

$$ u(x,t) = \sum_{m = -\infty}^{\infty} a_{m}(t) e^{j m x},$$

where $a_{m}(t)$ are complex time-dependent amplitudes of mode $m$, and $j$ is the imaginary unit. 

Since the advection equation is linear, we can consider each mode $a_{m}(t) e^{j m x}$ separately. The stability condition to guarantee that the solution stays finite is that the amplitude of all modes does not grow from one time step to the next:

$$ \Big| \frac{a_m(t_{n+1})}{a_m(t_n)}\Big| \le 1 \quad \forall m. $$

### FTCS stability
Let us study the stability of the FTCS scheme, with finite difference formulation:

$$ u_i^{n+1} = u_i^n - \frac{C}{2}(u^n_{i+1} - u^n_{i-1}).$$

Consider the positive $a$ case with left-to-right propagation.
Substituting the expression for the $m^{th}$ Fourier transform mode, we get:

$$ a_m(t_{n+1})e^{\,j mi\Delta x} = a_m(t_{n})e^{\,jmi\Delta x} - \frac{C}{2}\big(a_m(t_{n})e^{\,jm(i+1)\Delta x} - a_m(t_{n})e^{\,jm(i-1)\Delta x}\big),$$

where we used the expression $x_i = i\Delta x$ for the $i^{th}$ spatial grid point. Multiplying this equation by $e^{-jmi\Delta x}$, we obtain

$$ a_m(t_{n+1}) = a_m(t_{n}) - \frac{C}{2}\big(a_m(t_{n})e^{jm\Delta x} - a_m(t_{n})e^{-jm\Delta x}\big)$$

and thus the ratio:

$$\frac{a_m(t_{n+1})}{a_m(t_n)} = 1 - jC\sin{(m\Delta x)}.$$

The stability condition for the FTCS scheme can be checked by taking tha absolute value:

$$ \left | \frac{a_m(t_{n+1})}{a_m(t_n)} \right | = 1 + C^2 \sin ^2(m\Delta x). $$

Because this ratio is always greater than 1, the **FTCS scheme is unconditionally unstable** for any value of the time step and grid size.  

### Upwind stability

Using the same approach, we evaluate the stability of the upwind scheme:

$$ u_i^{n+1} = u_i^n - C(u^n_{i} - u^n_{i-1}).$$

Substituting the expression for the $m^{th}$ Fourier mode and multiplying by $e^{-jmi\Delta x}$, we obtain:

$$ a_m(t_{n+1}) = a_m(t_{n}) - C\big(a_m(t_{n}) - a_m(t_n)e^{-jm\Delta x}) $$

with amplitude ratio:

$$ \frac{a_m(t_{n+1})}{a_m(t_n)} = 1 - C + C e^{-jm\Delta x}. $$

The stability condition becomes

$$\left| \frac{a_m(t_{n+1})}{a_m(t_n)} \right| = 1-2C(1-C)(1-\cos(m\Delta x)) \le 1 \implies 2C (1-C) \le 0.$$ 

Our analysis leads to the following Upwind algorithm stability condition:

$$ C \le 1 .$$

Therefore, the **upwind scheme is conditionally stable**, requiring $C<1$ for stability.

## Lax methods

While biasing the derivative in the upwind direction is simple in one dimension, it is not always possible in two or more dimensions, especially when using unstructured grids. Two stable explicit schemes that allow one to use *centered derivatives* are the [Lax-Wendroff](https://en.wikipedia.org/wiki/Lax%E2%80%93Wendroff_method) and [Lax-Friedrichs](https://en.wikipedia.org/wiki/Lax%E2%80%93Friedrichs_method) approaches, both named after Peter Lax.<br> 

The Lax-Wendroff method, in particular, is a FTCS approach that is second-order accurate in both space and time, and is simple to implement. It adds to the FTCS scheme discussed above a dissipative term proportional to the second spatial derivative similar to the one we encountered in the parabolic diffusion equation:

$$ u_{i}^{{n+1}} = u_{i}^{n} - \frac{C}{2} \left( u_{{i+1}}^{{n}}-u_{{i-1}}^{{n}} \right) 
+ \frac{C^{2}}{2} \left( u_{{i+1}}^{{n}}-2u_{{i}}^{{n}}+u_{{i-1}}^{{n}}\right),$$

with $C= a \Delta t / \Delta x $. The addition of this term makes the FTCS scheme conditionally stable (for $C \le 1$) at the cost of including an artificial diffusion term that tends to broaden the initial distribution. In the context of the Vlasov and Boltzmann equations, advection is often accompanied by a collision term, which similarly modifies the solution and partially stabilizes the FTCS scheme.

## Predictor-corrector method

While the upwind method can provide a stable explicit scheme, there are several additional approaches for improving its accuracy. A possible improvement is to replace the Euler scheme with the 4th order Runge-Kutta method, which was described in a previous lecture. Here we will outline another approach: the predictor-corrector method. 

The idea of predictor-corrector (PC) is, first, to calculate $u(t + \Delta t)$ from $u(t)$ using an *explicit* method and then refine it using an _implicit_ method. The first step is called predictor and the second step corrector. In principle, one could apply a corrector step several times, but in practice, one corrector iteration is enough in most cases.

This combination of explicit and implicit methods gives a stable and computationally efficient solver. The simplest realization of a PC scheme is to use forward Euler as predictor and Backward Euler as corrector. For the advection equation, this scheme becomes:

1. *Predictor step:* Calculate $u_E(x,t + \Delta t)$ from $\dfrac{\partial u(x,t)}{\partial x}$ using forward Euler.<br>


2. *Corrector step:* Calculate $u(x,t + \Delta t)$ from $\dfrac{\partial u_E(x,t+\Delta t)}{\partial x}$ using backward Euler.

As the predictor-corrector technique consists of two or more steps, it is not possible to apply the Von Neumann stability analysis, although the stability can still be studied numerically. As a bonus content (courtesy of Dr. Ivan Maliyov), we implement below the PC approach for centered, forward, and backward spatial derivatives.

In [ ]:
# Solvers called by the PC routine
# FTFS solver
def ftfs(u, C):
    unew = np.zeros_like(u)
    # loop over space index from 1 to Nx-1
    for i in range(1,u.shape[0]-1):
        unew[i] = u[i] - C * (u[i+1]-u[i])
                        
    # Periodic boundary conditions
    unew[0] = u[0] - C * (u[1] - u[0])
    unew[-1] = unew[0]
    return unew

# FTBS solver
def ftbs(u, C):
    unew = np.zeros_like(u)
    # loop over space index from 1 to Nx-1
    for i in range(1,u.shape[0]-1):
        unew[i] = u[i] - C * (u[i]-u[i-1])
                        
    # Periodic boundary conditions
    unew[0] = u[0] - C * (u[0] - u[-2])
    unew[-1] = unew[0]
    return unew


In [ ]:
# Predictor-corrector solver
def predictor_corrector(u, C, space_der):
    
    # Centered space derivative
    if space_der == 'centered':
        
        # Predictor step 
        u_pred = ftcs(u,C)
        
        # Corrector step
        u_corr = np.zeros_like(u)
        # loop over space index from 1 to Nx-1
        for i in range(1,u.shape[0]-1):
            u_corr[i] = u[i] - C / 2.0 * (u_pred[i+1]-u_pred[i-1])
                        
        # Periodic boundary conditions
        u_corr[0] = u[0] - C / 2.0 * (u_pred[1] - u_pred[-2])
        u_corr[-1] = u_corr[0]
        return u_corr

    # Forward space derivative
    elif space_der == 'forward':
        
        # Predictor step
        u_pred = ftfs(u,C)
        
        # Corrector step
        u_corr = np.zeros_like(u)
        # loop over space index from 1 to Nx-1
        for i in range(1,u.shape[0]-1):
            u_corr[i] = u[i] - C * (u_pred[i+1]-u_pred[i])
                        
        # Periodic boundary conditions
        u_corr[0] = u[0] - C * (u_pred[1] - u_pred[0])
        u_corr[-1] = u_corr[0]
        return u_corr
    
    #Backward space derivative
    elif space_der == 'backward':
        
        # Predictor step
        u_pred = ftbs(u,C)
        
        # Corrector step
        u_corr = np.zeros_like(u)
        # loop over space index from 1 to Nx-1
        for i in range(1,u.shape[0]-1):
            u_corr[i] = u[i] - C * (u_pred[i]-u_pred[i-1])
                        
        # Periodic boundary conditions
        u_corr[0] = u[0] - C * (u_pred[0] - u_pred[-2])
        u_corr[-1] = u_corr[0]
        return u_corr
    
    # Unknown derivative
    else:
        sys.exit('Space derivative '+str(space_der)+' is unknown!')

# Initialization and initial condition for 
# the predictor-corrector with centered space derivative
u_pc_cs = np.zeros([Nt+1, Nx+1])
u_pc_cs[0,:] = initial_condition(x)

# Initialization and initial condition for 
# the predictor-corrector with forward space derivative
u_pc_fs = np.zeros([Nt+1, Nx+1])
u_pc_fs[0,:] = initial_condition(x)

# Initialization and initial condition for 
# the predictor-corrector with barckward space derivative
u_pc_bs = np.zeros([Nt+1, Nx+1])
u_pc_bs[0,:] = initial_condition(x)


# Iteration over time
for n in range(Nt):
    u_pc_cs[n+1,:] = predictor_corrector(u_pc_cs[n,:], C, 'centered')
    u_pc_fs[n+1,:] = predictor_corrector(u_pc_fs[n,:], C, 'forward')
    u_pc_bs[n+1,:] = predictor_corrector(u_pc_bs[n,:], C, 'backward')

# Root-mean-square errors
rmse_pc_cs = rmse(u_pc_cs,u_exact)
rmse_pc_fs = rmse(u_pc_fs,u_exact) # the forward space (downwind) derivative will diverge
rmse_pc_bs = rmse(u_pc_bs,u_exact)

Let us plot the RMSE values of the predictor-corrector approach with different spatial derivatives. 

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(time_array, rmse_pc_cs, label = 'PC-CS', color = color_dict['PC-CS'])
plt.plot(time_array, rmse_pc_fs, label = 'PC-FS', color = color_dict['PC-FS'])
plt.plot(time_array, rmse_pc_bs, label = 'PC-BS', color = color_dict['PC-BS'])
plt.xlabel('time')
plt.ylabel('RMSE')
plt.xlim([0,10])
plt.ylim([-1,1.5])
plt.legend(loc='upper right')
plt.show()

In the legend of the plot, PC stands for the predictor-corrector and the last two letters for the spatial derivative type: CS - Centered-Space; FS - Forward-Space; BS - Backward-Space.
We find that with the predictor-corrector approach two solvers are now stable: PC-BS and PC-CS. 
As one can check, the PC-BS gives almost the same solution as the upwind method, but the PC-CS produces a better result (lower RMSE value).

We compare below the solution $u(x,t)$ for a relatively large time $t=10$.

In [ ]:
plt.figure(figsize=(14,6))

ntime3 = int(10.0 / dt)

title = 'Time = '+str(time_array[ntime3])+'; C = '+str(C)

plt.plot(x, u_exact[ntime3,:], label = 'Exact', color = color_dict['Exact'])
plt.plot(x, u_pc_cs[ntime3,:], label = 'PC-CS', color = color_dict['PC-CS'])
plt.plot(x, u_pc_bs[ntime3,:], label = 'PC-BS', color = color_dict['PC-BS'])


#plt.plot(x,u_exact,label='Exact')
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('u')
plt.xlim([0, 1])
plt.title(title)

plt.tight_layout()
plt.show()

We can see that after a relatively long time, the curve of PC-CS, consistently with the RMSE results, is the closest to the exact solution. One should keep in mind though that the predictor-corrector method is at least twice more computationally expensive than the upwind method. 

## Summary

We implemented and compared different solvers for the advection equation, and analyzed the stability of the solutions first empirically and then using the Von Neumann method. 
We can draw the following conclusions:
1. The FTCS approach is unstable. Upwind method the simplest explicit solver for the advection equation. It is conditionally stable but not very accurate. 
2. The Von Neumann analysis is a helpful tool to determine the stability conditions of a numerical PDE solution method. Note that it can be applied only to relatively simple numerical schemes. 
3. The predictor-corrector technique is a useful and efficient approach for PDEs. However, the precise scheme should be adapted for the specific PDE. For the advection equation, we have found the following efficient scheme: Forward Euler as a predictor, backward Euler as corrector, and centered-space derivative.